In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import accelerate
import warnings
from pathlib import Path
import random
import re
import json
import numpy as np
import pandas as pd
import torch
import librosa
from transformers import TrainingArguments
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2FeatureExtractor
from datasets import load_metric
from transformers import Wav2Vec2ForCTC
from datasets import ClassLabel, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from IPython.display import Audio
from IPython.display import display, HTML



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
with open("../input/wolof-language/wolof/train/text") as file_name:
    data = file_name.read().split("\n")

In [39]:
audios_name = []
transcription_text = []

for i in data:
    audios_name.append(str(i[:18]))
    transcription_text.append(i[18:])

print(audios_name[857], transcription_text[857])

WOL_01_lect_0858   mu ngi ne rodd ci bopp koñ bi di xaar xaritam


In [40]:
audios_data = {
    "ID": audios_name,
    "transcription": transcription_text
}

In [41]:
wolof_train = pd.DataFrame(audios_data)
num_wolof_train = len(audios_name)
wolof_train = wolof_train[:num_wolof_train-1]
wolof_train.tail(5)


,ID,transcription
5992,WOL_09_lect_0995,ma nga ameewoon ca jamano yee ba rab yi daan w...
5993,WOL_09_lect_0996,amatuma dara xaaral ma seeti sama géeri juddu yi
5994,WOL_09_lect_0997,agsi na ci ngoon astafirlaa ci suba
5995,WOL_09_lect_0998,sa caalit ba ba nga gisee xaj ba lay nettali
5996,WOL_09_lect_0999,dama koy saam ci lay gëna gaawa jare


In [42]:
wolof_train.shape

(5997, 2)

In [43]:
# Dict with ID and paths of the audios
paths = {}
path = '../input/wolof-language/wolof/train'
for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith(".wav"):
      audio_path = os.path.join(root, file)
      paf = Path(audio_path)
      id = paf.parts[-1].split(".")[0]
      paths[id] = audio_path

In [44]:
wolof_train['file'] = [paths.get(id.strip(), '') for id in list(wolof_train['ID'])]
wolof_train_no_ID = wolof_train.drop(["ID"], axis = 1)
wolof_train_no_ID.head(5)

,transcription,file
0,sunu meetar ñówul tànkam dafa faxaj,../input/wolof-language/wolof/train/01/WOL_01_...
1,Nanga ma ko fàttali lu ko moy dinaa ko fàtte,../input/wolof-language/wolof/train/01/WOL_01_...
2,am na ñax mu nu daan foye xuusmaañàpp,../input/wolof-language/wolof/train/01/WOL_01_...
3,maamam a daan tëgg laxaab yi ca dëkk ba,../input/wolof-language/wolof/train/01/WOL_01_...
4,benn seex bi gàjj bi ci lex bi laa koy xàmmee,../input/wolof-language/wolof/train/01/WOL_01_...


In [45]:
def show_random_elements(dataset, n_examples):
  assert n_examples <= len(dataset), "Sorry, can't pick more elements than there are in the dataset."
  picks = []
  for i in range(n_examples):
    pick = random.randint(0, len(dataset)-1)
    while pick in picks:
      pick = random.randint(0, len(dataset)-1)
    picks.append(pick)

  df = pd.DataFrame(dataset[picks])
  display(HTML(df.to_html()))

In [46]:
chars_to_ignore_regex = '[\"\?\.\!\-\;\:\(\)\,]'
def remove_special_characters(batch):
  batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
  return batch

In [47]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [61]:
wolof = DatasetDict({"train": Dataset.from_pandas(wolof_train_no_ID[0:4798]), "test": Dataset.from_pandas(wolof_train_no_ID[4798:])})
wolof = wolof.map(remove_special_characters)

  0%|          | 0/4798 [00:00<?, ?ex/s]

  0%|          | 0/1199 [00:00<?, ?ex/s]

In [62]:
show_random_elements(wolof["train"], n_examples=5)

,transcription,file
0,abal ma mbàttu bu ma tibbe fonde bi,../input/wolof-language/wolof/train/07/WOL_07_lect_0658.wav
1,maa ngiy fàttaliku bés ba nu dajee foofa,../input/wolof-language/wolof/train/07/WOL_07_lect_0016.wav
2,xanaa yow amuloo dara kuy sebet benn peppu dugub,../input/wolof-language/wolof/train/07/WOL_07_lect_0274.wav
3,jinax yi dañu muus mënuloo leen posone,../input/wolof-language/wolof/train/07/WOL_07_lect_0367.wav
4,fekk na biram njéemé koo njaay a ngi wara tollook ndangalma,../input/wolof-language/wolof/train/02/WOL_02_lect_0609.wav


In [63]:
vocabs = wolof.map(extract_all_chars, batched = True, batch_size = -1, keep_in_memory = True, remove_columns = wolof.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [65]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}


In [66]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [67]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

35

In [68]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [69]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token = "[UNK]", pad_token = "[PAD]", word_delimiter_token = "|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size = 1, sampling_rate = 16000, padding_value = 0.0, do_normalize = True, return_attention_mask = True)
processor = Wav2Vec2Processor(feature_extractor = feature_extractor, tokenizer = tokenizer)

In [70]:
warnings.filterwarnings("ignore")

def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = librosa.load(batch["file"], sr = 16000)
  batch["speech"] = speech_array
  batch["sampling_rate"] = sampling_rate
  batch["target_text"] = batch["transcription"]
  return batch

In [71]:
wolof = wolof.map(speech_file_to_array_fn, remove_columns = wolof.column_names["train"], num_proc = 1)

  0%|          | 0/4798 [00:00<?, ?ex/s]

  0%|          | 0/1199 [00:00<?, ?ex/s]

In [72]:
def prepare_dataset(batch):
  assert (
      len(set(batch["sampling_rate"])) == 1
  ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
  batch["input_values"] = processor(batch["speech"], sampling_rate = batch["sampling_rate"][0]).input_values

  with processor.as_target_processor():
    batch["labels"] = processor(batch["target_text"]).input_ids
  return batch


In [73]:
wolof_prepared = wolof.map(prepare_dataset, remove_columns = wolof.column_names["train"], batch_size = 32, batched = True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

In [74]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [75]:
data_collator = DataCollatorCTCWithPadding(processor = processor, padding = True)


In [76]:
wer_metric = load_metric("wer")

In [77]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [78]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout = 0.0,
    hidden_dropout = 0.0,
    feat_proj_dropout = 0.0,
    mask_time_prob = 0.05,
    layerdrop = 0.0,
    ctc_loss_reduction = "mean",
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer),
)

    

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
model.freeze_feature_extractor()

In [80]:
torch.cuda.is_available()

True

In [81]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./customwolof",
    group_by_length = True,
    per_device_train_batch_size = 3,
    evaluation_strategy = "steps",
    num_train_epochs = 3,
    fp16 = True,
    save_steps = 40000,
    eval_steps = 1000,
    logging_steps = 1000,
    learning_rate = 1e-4,
    weight_decay = 0.005,
    warmup_steps = 1000,
    save_total_limit = 12,
)

In [82]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = wolof_prepared["train"],
    eval_dataset = wolof_prepared["test"],
    tokenizer = processor.feature_extractor,
)

In [83]:
trainer.train()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Wer
1000,3.670500,1.022015,0.812394


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Fatou/customWav2Vec2Model/commit/2cce517e97390afc395106e5d3bb9de559f0c456', commit_message='Upload tokenizer', commit_description='', oid='2cce517e97390afc395106e5d3bb9de559f0c456', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Push model to the Hub
trainer.push_to_hub("Fatou/customwolof")
tokenizer.push_to_hub("Fatou/customwolof")
